<a href="https://www.kaggle.com/code/bhavinmoriya/hypothesis-testing-2?scriptVersionId=111999204" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Chi square test

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from scipy.stats import norm, chisquare, chi2, chi2_contingency
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
population = pd.DataFrame(['white']*50000 + ['black'] * 100_000 + ['red'] * 25000 + ['asian'] * 40000 + ['others'] * 60000)
minnesota = pd.DataFrame(['white']*500 + ['black'] * 100 + ['red'] * 250 + ['asian'] * 400 + ['others'] * 600)

In [3]:
f_obs=minnesota.value_counts(normalize=False).sort_index()
f_exp=len(minnesota)*population.value_counts(normalize=True).sort_index()

In [4]:
minnesota.value_counts(normalize=True).sort_index(),population.value_counts(normalize=True).sort_index()

(asian     0.216216
 black     0.054054
 others    0.324324
 red       0.135135
 white     0.270270
 dtype: float64,
 asian     0.145455
 black     0.363636
 others    0.218182
 red       0.090909
 white     0.181818
 dtype: float64)

Dist is very different so we expect to reject null hyp.

In [5]:
statistic, pval = chisquare(f_obs=f_obs,
                            f_exp=f_exp)
print(f'Chi sq stat and pvalue are : {statistic, pval}')

Chi sq stat and pvalue are : (766.2162162162163, 1.5948364430109697e-164)


Hence at 5% sigficance lev we reject the null hyp and hence dist is different.

In [6]:
crit = chi2.ppf(q=.95, # 95% conf level
               df=4) # deg of freedom is no of categories - 1
pval = 1 - chi2.cdf(x=statistic,
                   df = 4)
print(f'Critical and P values are : {crit, pval}')

Critical and P values are : (9.487729036781154, 0.0)


Since statistic value is way above critical value we reject the NH. One more reason to reject NH is, pval is < .05 (5% sig lev).

# Chi square test for independence

What if we wanted to check if two distribution? For example,

1. Birthdate might not tell if you like rice or not.
2. Eduacation might not tell what toothpaste you use.

So two check the independent dist we use this test. We shall check the relationship b/w race and political viewpoint.

In [7]:
np.random.seed(0)
voter_race = np.random.choice(a='asian black white red others'.split(),
                             p = [.3, .3, .1, .2, .1],
                             size=1000)
voter_party = np.random.choice(a='democrat independent republican'.split(),
                             p = [.8, .1, .1],
                             size=1000)

In [8]:
voters = pd.crosstab(voter_race, voter_party, margins=True)
observed = voters.iloc[:-1,:-1]
observed

col_0,democrat,independent,republican
row_0,,,
asian,240,28,39
black,242,31,30
others,94,5,9
red,128,25,30
white,71,16,12


Now we calculate expected proportions which is last column producting with last row and normalize with 1000 (size of data).

In [9]:
expected = pd.DataFrame((np.outer(voters.loc[:,'All'],
                                  voters.loc['All',:]) / 1000)[:-1, :-1])
expected.index = voters.index[:-1]
expected.columns = voters.columns[:-1]

expected.index.name = 'Race'
expected.columns.name = 'Party'

expected

Party,democrat,independent,republican
Race,,,
asian,237.925,32.235,36.84
black,234.825,31.815,36.36
others,83.700,11.340,12.96
red,141.825,19.215,21.96
white,76.725,10.395,11.88


In [10]:
chi2_contingency(observed=observed)

(17.559352010250556,
 0.024783918488562488,
 8,
 array([[237.925,  32.235,  36.84 ],
        [234.825,  31.815,  36.36 ],
        [ 83.7  ,  11.34 ,  12.96 ],
        [141.825,  19.215,  21.96 ],
        [ 76.725,  10.395,  11.88 ]]))

In [11]:
stat, pval, _, _ = chi2_contingency(observed=observed)
crit = chi2.ppf(q=.95,
               df=8) # product of no of column - 1 with no of rows - 1
pval2 = 1 - chi2.cdf(x=stat,
                df=8)
#pval2 would be same as pval
print(f'Critical value, pval and stat are : {crit, pval, pval2, stat}')

Critical value, pval and stat are : (15.50731305586545, 0.024783918488562488, 0.02478391848856254, 17.559352010250556)


1. Stats is above Critical value $\Rightarrow$ we reject NH. Hence data is dependent.
2. pvalue < .05 $\Rightarrow$ we reject NH. Hence data is dependent.
